In [ ]:
import rasterio
import affine
from rasterio import plot
import numpy as np
import shapefile
from math import sqrt
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
np.seterr(divide='ignore', invalid='ignore') #Il se peut qu'il y ait des divisions par 0, numpy retourne 0 dans ces cas.


In [ ]:
IMAGE_PIXEL = 30
DEFAULT = "not_labeled"
S1DIRECTORY = "output/sentinel1"
output_filename = 'tile_'

Pour le bon fonctionnement des entrainements et des prédictions, les fichiers passés ci-dessous doivent avoir été harmonisés par le notebook TranslateS1Bands.

In [ ]:
springvh = rasterio.open('S1Spring/sp_vh.tiff') 
springvv = rasterio.open('S1Spring/sp_vv.tiff')
summervh = rasterio.open('S1Summer/su_vh.tiff') 
summervv = rasterio.open('S1Summer/su_vv.tiff')
fallvh = rasterio.open('S1Fall/fa_vh.tiff') 
fallvv = rasterio.open('S1Fall/fa_vv.tiff')
wintervh = rasterio.open('S1Winter/wi_vh.tiff') 
wintervv = rasterio.open('S1Winter/wi_vv.tiff')

In [ ]:
from osgeo import gdal
def create_gcps(fall, winter, spring, summer):
    """Fait une approximation des gcps en moyennant les gcps de toutes les bandes de saisons (prend comme point de départ le
    plus petit x et le plus petit y trouvé)"""
    x_min = min([fall[0].x, winter[0].x, spring[0].x, summer[0].x])
    y_min = min([fall[0].y, winter[0].y, spring[0].y, summer[0].y])
    gcps = [rasterio.control.GroundControlPoint(row=0.0, col=0.0, x=x_min, y=y_min, id='1')]
    for i in range(1, len(fall)):
        previous_x_average = (fall[i-1].x + winter[i-1].x + spring[i-1].x + summer[i-1].x) / 4
        previous_y_average = (fall[i-1].y + winter[i-1].y + spring[i-1].y + summer[i-1].y) / 4
        actual_x_average = (fall[i].x + winter[i].x + spring[i].x + summer[i].x) / 4
        actual_y_average = (fall[i].y + winter[i].y + spring[i].y + summer[i].y) / 4
        x = gcps[i-1].x + actual_x_average - previous_x_average
        y = gcps[i-1].y + actual_y_average - previous_y_average
        gcps.append(rasterio.control.GroundControlPoint(row=fall[i].row, col=fall[i].col, x=x, y=y, id=str(i+1)))
    return gcps

In [ ]:
def get_vectors(gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left):
    """Forme les vecteurs moyens entre 4 coordonnées GroundControlPoint"""
    norm_v1 = (gcp_top_left.col - gcp_top_right.col)
    v1 = np.array([(gcp_top_left.x - gcp_top_right.x + gcp_bottom_left.x - gcp_bottom_right.x) / (2 * norm_v1),
         (gcp_top_left.y - gcp_top_right.y + gcp_bottom_left.y - gcp_bottom_right.y) / (2 * norm_v1)])
    norm_v2 = (gcp_bottom_right.row - gcp_top_right.row)
    v2 = np.array([(gcp_bottom_right.x - gcp_top_right.x + gcp_bottom_left.x - gcp_top_left.x) / (2 * norm_v2),
         (gcp_bottom_right.y - gcp_top_right.y + gcp_bottom_left.y - gcp_top_left.y ) / (2 * norm_v2)])
    return v1, v2

In [ ]:
from sympy import symbols, Eq, solve
def get_offset(gcps, x_min, y_min):
    """Retourne déplacements à effectuer pour avoir la position d'une coordonnée particulière (environ)"""
    gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left = None, None, None, None
    for gcp in gcps:
        if gcp.row == 0.0 and gcp.col == 0.0:
            gcp_top_right = gcp
        elif gcp.row == 0.0 and gcp_top_left == None:
            gcp_top_left = gcp
        elif gcp.col <= 0.0 and gcp_bottom_right == None:
            gcp_bottom_right = gcp
        elif gcp_bottom_right != None:
            gcp_bottom_left = gcp
            break
    v1, v2 = get_vectors(gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left)
    origin = np.array([gcp_top_right.x, gcp_top_right.y])
    u1, u2 = symbols('u1 u2')
    eq1 = Eq(gcp_top_right.x - u1*v1[0] - u2*v2[0] - x_min, 0)
    eq2 = Eq(gcp_top_right.y - u1*v1[1] - u2*v2[1] - y_min, 0)
    sol = solve((eq1, eq2),(u1, u2))

    return round(sol[u1]), round(sol[u2])

In [ ]:
def translate_image(image, offset_x, offset_y):
    """Fait correspondre la position [0][0] de notre image à la position [offset_y][offset_x], toutes positions en dehors de
    l'image de base seront mises à 0."""
    result = []
    for i in range(len(image)):
        if 0 <= (i + offset_y) < len(image):
            result_row = []
            for j in range(len(image[0])):
                if 0 <= (j + offset_x) < len(image[0]):
                    result_row.append(image[i + offset_y][j + offset_x])
                else:
                    result_row.append(0)
            result.append(result_row)
        else:
            result.append([0 for i in range(len(image[0]))])
    return np.array(result, dtype=np.uint16)

In [ ]:
def make_blue(vv, vh):
    #Sources https://stackoverflow.com/questions/3950372/round-with-integer-division
    return (vv + vh // 2) // vh 

In [ ]:
def split_band(band, length, width):
    """Split une bande de Sentinel-1 en bande plus petite"""
    result = []
    for x in range(length):
        columns = []
        for y in range(width):
            lines = []
            for i in range(IMAGE_PIXEL):
                cells = []
                for j in range(IMAGE_PIXEL):
                    cells.append(band[i + IMAGE_PIXEL * x][j + IMAGE_PIXEL * y])
                lines.append(cells)
            columns.append(lines)
        result.append(columns)
    return result

In [ ]:
from rasterio import transform
def create_temp_image(springvh, springvv, summervh, summervv, fallvh, fallvv, wintervh, wintervv, name="temp"):
    """Créée des images rgb avant le découpage en image plus petites"""
    
    gcps_sp, gcp_crs = springvv.gcps
    gcps_su, _ = summervv.gcps
    gcps_fa, _ = fallvv.gcps
    gcps_wi, _ = wintervv.gcps
    
    gcps = create_gcps(gcps_fa, gcps_wi, gcps_sp, gcps_su)
    
    temp = rasterio.open('output/' + str(name) + '.tiff', 'w', driver='Gtiff', 
                          width=len(springvh.read(1)[0]), height=len(springvh.read(1)), count=12, crs=gcp_crs, gcps=gcps,
                          transform=springvh.transform,
                          dtype=springvh.dtypes[0]
                         )
    
    
    temp.write(make_blue(np.array(summervv.read(1)), np.array(summervh.read(1))), 12)
    temp.write(summervh.read(1), 11)
    temp.write(summervv.read(1), 10)
    temp.write(make_blue(np.array(springvv.read(1)), np.array(springvh.read(1))), 9)
    temp.write(springvh.read(1), 8)
    temp.write(springvv.read(1), 7)
    temp.write(make_blue(np.array(wintervv.read(1)), np.array(wintervh.read(1))), 6)
    temp.write(wintervh.read(1), 5)
    temp.write(wintervv.read(1), 4)
    temp.write(make_blue(np.array(fallvv.read(1)), np.array(fallvh.read(1))), 3)
    temp.write(fallvh.read(1), 2)
    temp.write(fallvv.read(1), 1)
    temp.close()
    print("Fichier tiff créé.")

In [ ]:
from rasterio import transform
_, gcp_crs = springvv.gcps
dtype = springvh.dtypes[0]
transform = springvh.transform

create_temp_image(springvh, springvv, summervh, summervv, fallvh, fallvv, 
                 wintervh, wintervv,name="Temp")

In [ ]:
import gdal
#source https://gis.stackexchange.com/questions/14712/splitting-raster-into-smaller-chunks-using-gdal
in_file = 'output/Temp.tiff'

ds = gdal.Open(in_file)
band = ds.GetRasterBand(1)

xsize = band.XSize
ysize = band.YSize
if not os.path.isdir(S1DIRECTORY):
    os.mkdir(S1DIRECTORY)
if not os.path.isdir(S1DIRECTORY + "/" + DEFAULT):
        os.mkdir(S1DIRECTORY + "/" + DEFAULT)
for i in range(0, ysize, IMAGE_PIXEL):
    if not os.path.isdir(S1DIRECTORY + "/" + DEFAULT + "/" + str(int(i/IMAGE_PIXEL))):
        os.mkdir(S1DIRECTORY + "/" + DEFAULT + "/" + str(int(i/IMAGE_PIXEL)))
        
    print(i)
    for j in range(0, xsize, IMAGE_PIXEL):
        com_string = "gdal_translate -of GTIFF -srcwin " + str(j) + ", " + str(i) + ", " + str(IMAGE_PIXEL) + \
            ", " + str(IMAGE_PIXEL) + " " + str(in_file) + " " + str(S1DIRECTORY) + "/" + DEFAULT + "/" + \
            str(int(i/IMAGE_PIXEL)) + "/" + str(output_filename) + str(int(i/IMAGE_PIXEL)) + "_" + \
            str(int(j/IMAGE_PIXEL)) + ".tiff"
        os.system(com_string)

In [ ]:
import shapefile
import math

In [ ]:
LIST_SHAPEFILES = ["heigvd/central_highlands_2_other/central_highlands_2_other.shp",
                  "heigvd/central_highlands_2_test/central_highlands_2_test.shp",
                  "heigvd/central_higlands_1_other/central_higlands_1_other.shp"]
CORRESPONDANCES = {"1":"cacao", "2":"coffee","3":"complex_oil", "4":"nativevege", "5":"oil_palm", "6":"rubber", "7":"unknown", 
                 "8":"seasonal", "9":"urban", "10":"water", "11":"other_tree", "12":"other_no_tree", "13":"native_no_tree",
                 "14":"water_other", "15":"pepper", "16":"cassava", "17":"tea", "18":"rice", "19":"banana", "20":"baby_palm", 
                 "21":"cur_off-regrow", "22":"natural_wetland", "23":"intercrop", "24":"deciduous_forest", "25":"stick_pepper", 
                 "26":"flooded_plantation", "27":"pine_trees", "28":"coconut", "29":"banboo", "30":"savana", "31":"mango", 
                 "32":"other_fruit_tree_crop", "33":"water_mine", "0":"not_labeled", "-1":"ambiguous"}

In [ ]:
points = []
for path in LIST_SHAPEFILES:
    sf = shapefile.Reader(path)
    shapes = sf.shapes()
    for point in sf.records():
        points.append((shapes[point.oid].points[0], point[0]))

Code permettant l'interpolation dans un quadrilatère quelconque.

In [ ]:
def XY_to_LM_init(px, py):
    """Crée deux vecteurs servant à voir si une coordonnée se trouve dans l'image ou non (réutilisation du labo5 de VTK)"""
    A = np.array([[1, 0, 0, 0],
                  [1, 1, 0, 0],
                  [1, 1, 1, 1],
                  [1, 0, 1, 0]])
    AI = np.linalg.inv(A)
    a = np.dot(AI, px)
    b = np.dot(AI, py)
    return a, b

In [ ]:
def XY_to_LM(latitude, longitude, a, b):
    """Retourne l et m, si l et m sont entre 0 et 1 alors la coordonnée donnée est dans le
    carré (réutilisation du labo5 de VTK)"""
    aa = a[3] * b[2] - a[2] * b[3]
    bb = a[3] * b[0] - a[0] * b[3] + a[1] * b[2] - a[2] * b[1] + longitude * b[3] - latitude * a[3]
    cc = a[1] * b[0] - a[0] * b[1] + longitude * b[1] - latitude * a[1]
    det = math.sqrt(bb * bb - 4 * aa * cc)
    m = (-bb + det) / (2 * aa)
    l = (longitude - a[0] - a[2] * m) / (a[1] + a[3] * m)
    return l, m

In [ ]:
def get_corners(image):
    gcps, _ = image.gcps
    #Calcul des gcps voisins
    gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left = None, None, None, None
    for gcp in gcps:
        if gcp.row <= 0 and gcp.col <= 0:
            gcp_top_right = gcp
        elif gcp.row <= 0 and gcp.col >= IMAGE_PIXEL and (gcp_top_left == None or gcp_top_left.col >= gcp.col):
            gcp_top_left = gcp
        elif gcp.row >= IMAGE_PIXEL and gcp.col <= 0 and (gcp_bottom_right == None or gcp_bottom_right.row >= gcp.row):
            gcp_bottom_right = gcp
        elif gcp.row >= IMAGE_PIXEL and gcp.col >= IMAGE_PIXEL:
            gcp_bottom_left = gcp
            break
        
    #Pour les cas en bordure
    if gcp_bottom_left == None:
        gcp_bottom_left = gcps[-1]
    if gcp_bottom_right == None or gcp_top_left == None:
        for gcp in gcps:
            if gcp.row == gcp_top_right.row and gcp.col == gcp_bottom_left.col:
                gcp_top_left = gcp
            elif gcp.row == gcp_bottom_left.row and gcp.col == gcp_top_right.col:
                gcp_bottom_right = gcp
            if gcp_bottom_right != None and gcp_top_left != None:
                break
    #Calcul des vecteurs
    v1, v2 = get_vectors(gcp_top_right, gcp_top_left, gcp_bottom_right, gcp_bottom_left)
    #Calcul des coins
    top_right = (np.array([gcp_top_right.x, gcp_top_right.y]) - gcp_top_right.col * v1) - gcp_top_right.row * v2
    top_left = top_right + IMAGE_PIXEL * v1
    bottom_right = top_right + IMAGE_PIXEL * v2
    bottom_left = bottom_right + IMAGE_PIXEL * v1
    return {"tr":top_right, "tl":top_left, "br":bottom_right, "bl":bottom_left}

In [ ]:
def get_labels(image):
    """Donne tous les labels contenu dans une image de Sentinel-1"""
    corners = get_corners(image)
    labels = set()
    px = np.array([corners["bl"][0], corners["br"][0], corners["tr"][0], corners["tl"][0]])
    py = np.array([corners["bl"][1], corners["br"][1], corners["tr"][1], corners["tl"][1]])
    a, b = XY_to_LM_init(px, py)
    #A cause de la précision des floats, il arrive bien souvent que le dernier vecteur soit mis à 0
    #ce qui rend la localisation infaisable (retournera nan au lieu d'un float)
    if a[3] != 0 and b[3] != 0: 
        for point in points:
            l, m = XY_to_LM(point[0][1], point[0][0], a, b)
            if 0 <= l <= 1 and 0 <= m <= 1:
                labels.add(point[1])
    else: #Si problème de précision, on fait une approximation
        for point in points:
            if ((corners["bl"][0] + corners["tl"][0]) / 2) < point[0][0] < ((corners["br"][0] + corners["tr"][0]) / 2) and \
                ((corners["bl"][1] + corners["br"][1]) / 2) < point[0][1] < ((corners["tl"][1] + corners["tr"][1]) / 2):
                labels.add(point[1])
    return labels

In [ ]:
for _, directory in CORRESPONDANCES.items():
    if not os.path.isdir(S1DIRECTORY + "/" + directory):
        os.mkdir(S1DIRECTORY + "/" + directory)

In [ ]:
for directory in os.listdir(S1DIRECTORY + "/" + DEFAULT):
    if os.path.isdir(S1DIRECTORY + "/" + DEFAULT + "/" + directory):
        print(directory)
        for file in os.listdir(S1DIRECTORY + "/" + DEFAULT + "/" + directory):
            if file.endswith(".tiff") and os.path.isfile(S1DIRECTORY + "/" + DEFAULT + "/" + directory + "/" + file):
                labels = get_labels(rasterio.open(S1DIRECTORY + "/" + DEFAULT + "/" + directory + "/" + file))
                if len(labels) == 1:
                    os.rename(S1DIRECTORY + "/" + DEFAULT + "/" + directory + "/" + file, 
                              S1DIRECTORY + "/" + CORRESPONDANCES[str(next(iter(labels)))] + "/" + file) 
                elif len(labels) > 1:
                    if len(labels) > 10:
                        print("Error")
                        
                    os.rename(S1DIRECTORY + "/" + DEFAULT + "/" + directory + "/" + file, 
                              S1DIRECTORY + "/" + CORRESPONDANCES["-1"] + "/" + file)